# Протоколы транспортировки ключей

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

----

In [3]:
# КГПСЧ
from Crypto.Random import get_random_bytes

# Шифрование 
from Crypto.Cipher import AES

# Идентификаторы клиентов
import uuid

# Утилитки
from Crypto.Util import number
import binascii

-----

In [4]:
def bytes_as_hex(b: bytes) -> str:
    """
    Перевод бинарных данных в hex-строчку
    """
    
    return binascii.hexlify(b).decode()

----

## Протокол Нидхема-Шрёдера
![Needham-Schroeder Protocol](./images/Needham-Schroeder_3.png)

In [5]:
class NeedhamSchroederParams(object):
    SESSION_KEY_SIZE    = AES.key_size[2]
    ENCRYPTION_KEY_SIZE = AES.key_size[2]
    NUMBER_SIZE         = 4

In [6]:
class NeedhamSchroederCA(object):
    def __init__(self):
        """
        Инициализация доверенного центра.
        Создание пустой базы.
        """
        
        self._db = {}
        
    
    def register_client(self, client_id, client_key):
        """
        Регистрация клиента по его идентификатору и ключу.
        Если клиент уже есть, то бросает исключение.
        """
        
        trace('[NeedhamSchroederCA]', f'''Attempting to register client {client_id} 
        with key {bytes_as_hex(client_key)}''')
        
        if client_id in self._db:
            trace('[NeedhamSchroederCA]', f'Client {client_id} already exists')
            raise ValueError(f'Client {client_id} already exists')
            
        self._db[client_id] = client_key
        trace('[NeedhamSchroederCA]', 'Client registered successfully')
        
        
    def accept_initiation(self, first_id: uuid.UUID, second_id: uuid.UUID, number: bytes) -> bytes:
        """
        Выполнение функции ДЦ в процессе транспортировки ключа.
        """
        
        trace('[NeedhamSchroederCA]', f'Accepted initiation from {first_id} to {second_id}, Na = {bytes_as_hex(number)}')
        
        if first_id not in self._db or second_id not in self._db:
            trace('[NeedhamSchroederCA]', f"Client doesn't exist")
            raise ValueError(f"Client doesn't exist")
            
        #
        # Случайный ключ создаю
        #
        
        session_key = get_random_bytes(NeedhamSchroederParams.SESSION_KEY_SIZE)
        
        #
        # Зашифровываю внутренний пакет
        #
        
        cipher = AES.new(self._db[second_id], AES.MODE_CTR)
        nonce, internal_packet = cipher.nonce, cipher.encrypt(session_key + first_id.bytes)
        
        #
        # А теперь зашифровываю весь пакет
        #
        
        cipher = AES.new(self._db[first_id], AES.MODE_CTR)
        return cipher.nonce, cipher.encrypt(number + second_id.bytes + session_key + nonce + internal_packet)

In [7]:
class NeedhamSchroederClient(object):
    def __init__(self, fake_data: uuid.UUID = None):
        """
        Инициализация клиента с созданием идентификатора и генерацией ключа.
        Создается пустая база для сессионных ключей и челленджей.
        Второй параметр требуется для демонстрации неуспешной попытки исполнения протокола.
        """
        
        self._id  = uuid.uuid4() if fake_data is None else fake_data
        self._key = get_random_bytes(NeedhamSchroederParams.ENCRYPTION_KEY_SIZE)
        
        self._db  = {}
        self._challenges = {}
        
        trace('[NeedhamSchroederClient]', f'Client {self._id} created')
        
        
    def register(self, ca: NeedhamSchroederCA):
        """
        Регистрация на ДЦ.
        """
        
        ca.register_client(self._id, self._key)
        
        
    def initiate_protocol(self, second_client, ca: NeedhamSchroederCA) -> bool:
        """
        Инициация протокола транспортировки ключа. Принимает дескрипторы
        второго клиента и доверенного центра.
        Возвращает статус успешности выполнения протокола.
        """
        
        N = get_random_bytes(NeedhamSchroederParams.NUMBER_SIZE)
        trace('[NeedhamSchroederClient]', f'[{self._id}] N = {bytes_as_hex(N)}')
        
        nonce, encrypted_data = ca.accept_initiation(self._id, second_client.identifier, N)
        
        cipher = AES.new(self._key, AES.MODE_CTR, nonce=nonce)
        packet = cipher.decrypt(encrypted_data)
        
        if packet.find(N) != 0:
            trace('[NeedhamSchroederClient]', f'[{self._id}] Malformed response')
            return False
            
        #
        # Отрезаю лишнее и отправляю второму участнику нужные данные
        #
        
        packet = packet[len(N) + len(second_client.identifier.bytes):]
        session_key = packet[:NeedhamSchroederParams.SESSION_KEY_SIZE]
        
        #
        # Вычленяю nonce и шифртекст
        # Про размер nonce: If not present, the library creates a random nonce of length equal to block size/2.
        #
        
        packet = packet[NeedhamSchroederParams.SESSION_KEY_SIZE:]
        nonce, internal_packet = packet[:AES.block_size // 2], packet[AES.block_size // 2:]
        
        #
        # Получаю челлендж, модифицирую его и отправляю на проверку
        #
        
        challenge = second_client._accept_packet(nonce, internal_packet, self._id)
        if challenge is None:
            trace('[NeedhamSchroederClient]', f'[{self._id}] Second client denied packet')
            return False
        
        nonce, challenge = challenge
        cipher = AES.new(session_key, AES.MODE_CTR, nonce=nonce)
        
        challenge = cipher.decrypt(challenge)
        response = NeedhamSchroederClient._modify_challenge(challenge)
        
        cipher = AES.new(session_key, AES.MODE_CTR)
        if not second_client._verify_response(cipher.nonce, cipher.encrypt(response), self._id):
            trace('[NeedhamSchroederClient]', f'[{self._id}] Second client denied response')
            return False
        
        #
        # Проверки прошли - ключ общий есть
        #
        
        self._db[second_client.identifier] = session_key
        trace('[NeedhamSchroederClient]', f'[{self._id}] Established key: {bytes_as_hex(session_key)}')
        
        return True
    
    
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора клиента.
        """
        
        return self._id
        
        
    def _accept_packet(self, nonce: bytes, internal_packet: bytes, first_id: uuid.UUID):
        cipher = AES.new(self._key, AES.MODE_CTR, nonce=nonce)
        packet = cipher.decrypt(internal_packet)
        
        session_key = packet[:NeedhamSchroederParams.SESSION_KEY_SIZE]
        identifier  = packet[NeedhamSchroederParams.SESSION_KEY_SIZE:]
        
        #
        # Сравниваю идентификаторы
        #
        
        if identifier != first_id.bytes:
            trace('[NeedhamSchroederClient]', f'[{self._id}] Wrong identifier')
            return None
        
        #
        # Все ок, формирую челлендж
        #
        
        self._db[first_id] = session_key
        self._challenges[first_id] = get_random_bytes(NeedhamSchroederParams.NUMBER_SIZE)
        
        trace('[NeedhamSchroederClient]', f'[{self._id}] Generated challenge {bytes_as_hex(self._challenges[first_id])}')
        
        cipher = AES.new(session_key, AES.MODE_CTR)
        return cipher.nonce, cipher.encrypt(self._challenges[first_id])
    
    
    def _verify_response(self, nonce: bytes, response: bytes, first_id: uuid.UUID) -> bool:
        cipher = AES.new(self._db[first_id], AES.MODE_CTR, nonce=nonce)
        
        response = cipher.decrypt(response)
        expected = NeedhamSchroederClient._modify_challenge(self._challenges[first_id])
        
        trace('[NeedhamSchroederClient]', f'[{self._id}] response = {bytes_as_hex(response)}')
        trace('[NeedhamSchroederClient]', f'[{self._id}] expected = {bytes_as_hex(expected)}')
        
        #
        # Если ответ совпал с ожидаемым, то все чудесно
        #
        
        result = expected == response
        if result:
            trace('[NeedhamSchroederClient]', f'[{self._id}] Established key: {bytes_as_hex(self._db[first_id])}')
            
        return result
    
    
    @staticmethod
    def _modify_challenge(challenge: bytes) -> bytes:
        num = number.bytes_to_long(challenge)
        return number.long_to_bytes(num - 1)

---

In [8]:
# Создаю доверенный центр
ca = NeedhamSchroederCA()

In [9]:
# Два хороших пользователя
alice = NeedhamSchroederClient()
bob   = NeedhamSchroederClient()

[TRACE] [NeedhamSchroederClient] Client 743a108a-e29f-4c69-8bf3-2ce2cc797ac3 created
[TRACE] [NeedhamSchroederClient] Client 28529241-a909-440a-aa1e-963db0c6bb9d created


In [10]:
# Регистрация валидных пользователей
alice.register(ca)
bob.register(ca)

[TRACE] [NeedhamSchroederCA] Attempting to register client 743a108a-e29f-4c69-8bf3-2ce2cc797ac3 
        with key bd9a8adb2aa682499bcf31c56e6aa9d0b5033a2b469e854717979c2a60557036
[TRACE] [NeedhamSchroederCA] Client registered successfully
[TRACE] [NeedhamSchroederCA] Attempting to register client 28529241-a909-440a-aa1e-963db0c6bb9d 
        with key 61a183cdc84ce0e546cee38d100b743cab273a557a73ed197442599bdeb1fe30
[TRACE] [NeedhamSchroederCA] Client registered successfully


In [11]:
# Протокол чудесно работает для валидных пользователей
alice.initiate_protocol(bob, ca)

[TRACE] [NeedhamSchroederClient] [743a108a-e29f-4c69-8bf3-2ce2cc797ac3] N = f41297d8
[TRACE] [NeedhamSchroederCA] Accepted initiation from 743a108a-e29f-4c69-8bf3-2ce2cc797ac3 to 28529241-a909-440a-aa1e-963db0c6bb9d, Na = f41297d8
[TRACE] [NeedhamSchroederClient] [28529241-a909-440a-aa1e-963db0c6bb9d] Generated challenge 9387ccdd
[TRACE] [NeedhamSchroederClient] [28529241-a909-440a-aa1e-963db0c6bb9d] response = 9387ccdc
[TRACE] [NeedhamSchroederClient] [28529241-a909-440a-aa1e-963db0c6bb9d] expected = 9387ccdc
[TRACE] [NeedhamSchroederClient] [28529241-a909-440a-aa1e-963db0c6bb9d] Established key: dd83c0258a2d5b6027f5afec6cb64457fd883015f6993ea2c22f7a07dc1f4e21
[TRACE] [NeedhamSchroederClient] [743a108a-e29f-4c69-8bf3-2ce2cc797ac3] Established key: dd83c0258a2d5b6027f5afec6cb64457fd883015f6993ea2c22f7a07dc1f4e21


True

In [12]:
# А это злоумышленник
mallory = NeedhamSchroederClient(bob.identifier)

[TRACE] [NeedhamSchroederClient] Client 28529241-a909-440a-aa1e-963db0c6bb9d created


In [13]:
# Он не может получить ключ, если влезет в общение вместо Боба
alice.initiate_protocol(mallory, ca)

[TRACE] [NeedhamSchroederClient] [743a108a-e29f-4c69-8bf3-2ce2cc797ac3] N = f1192434
[TRACE] [NeedhamSchroederCA] Accepted initiation from 743a108a-e29f-4c69-8bf3-2ce2cc797ac3 to 28529241-a909-440a-aa1e-963db0c6bb9d, Na = f1192434
[TRACE] [NeedhamSchroederClient] [28529241-a909-440a-aa1e-963db0c6bb9d] Wrong identifier
[TRACE] [NeedhamSchroederClient] [743a108a-e29f-4c69-8bf3-2ce2cc797ac3] Second client denied packet


False

In [14]:
# Также он не сможет получить ключ, если сам инициирует протокол с Алисой
mallory.initiate_protocol(alice, ca)

[TRACE] [NeedhamSchroederClient] [28529241-a909-440a-aa1e-963db0c6bb9d] N = 6d2c9d5d
[TRACE] [NeedhamSchroederCA] Accepted initiation from 28529241-a909-440a-aa1e-963db0c6bb9d to 743a108a-e29f-4c69-8bf3-2ce2cc797ac3, Na = 6d2c9d5d
[TRACE] [NeedhamSchroederClient] [28529241-a909-440a-aa1e-963db0c6bb9d] Malformed response


False

----

## Протокол Отвея-Рииза
![Otway-Rees Protocol](./images/Otway-Rees_3.png)

In [15]:
class OtwayReesParams(object):
    SESSION_KEY_SIZE    = AES.key_size[2]
    ENCRYPTION_KEY_SIZE = AES.key_size[2]
    NUMBER_SIZE         = 4

In [16]:
class OtwayReesCA(object):
    def __init__(self):
        """
        Инициализация доверенного центра.
        Создание пустой базы.
        """
        
        self._db = {}
        
    
    def register_client(self, client_id, client_key):
        """
        Регистрация клиента по его идентификатору и ключу.
        Если клиент уже есть, то бросает исключение.
        """
        
        trace('[OtwayReesCA]', f'''Attempting to register client {client_id} 
        with key {bytes_as_hex(client_key)}''')
        
        if client_id in self._db:
            trace('[OtwayReesCA]', f'Client {client_id} already exists')
            raise ValueError(f'Client {client_id} already exists')
            
        self._db[client_id] = client_key
        trace('[OtwayReesCA]', 'Client registered successfully')
        
        
    def accept(self, N: bytes, first_client, second_client, 
               first_nonce: bytes, first_encrypted_packet: bytes, 
               second_nonce: bytes, second_encrypted_packet: bytes):
        """
        Прием сообщений от клиента. Производит проверки зашифрованных пакетов
        и возвращает зашифрованный на ключах пользователей сессионный ключ.
        """
        
        trace('[OtwayReesCA]', f'''Accepted initiation from {first_client.identifier} to {second_client.identifier}, N = {bytes_as_hex(N)}''')
        
        cipher = AES.new(self._db[first_client.identifier], AES.MODE_CTR, nonce=first_nonce)
        first_packet = cipher.decrypt(first_encrypted_packet)
        
        Na, first_packet = first_packet[:OtwayReesParams.NUMBER_SIZE], first_packet[OtwayReesParams.NUMBER_SIZE:]
        N1, first_packet = first_packet[:OtwayReesParams.NUMBER_SIZE], first_packet[OtwayReesParams.NUMBER_SIZE:]
        
        #
        # Проверю идентификаторы
        #
        
        if first_client.identifier.bytes != first_packet[:len(first_client.identifier.bytes)]:
            trace('[OtwayReesCA]', 'Malformed packet')
            return None
        
        if second_client.identifier.bytes != first_packet[len(first_client.identifier.bytes):]:
            trace('[OtwayReesCA]', 'Malformed packet')
            return None
        
        if N != N1:
            trace('[OtwayReesCA]', 'Malformed packet')
            return None
        
        cipher = AES.new(self._db[second_client.identifier], AES.MODE_CTR, nonce=second_nonce)
        second_packet = cipher.decrypt(second_encrypted_packet)
        
        Nb, second_packet = second_packet[:OtwayReesParams.NUMBER_SIZE], second_packet[OtwayReesParams.NUMBER_SIZE:]
        N2, second_packet = second_packet[:OtwayReesParams.NUMBER_SIZE], second_packet[OtwayReesParams.NUMBER_SIZE:]
        
        #
        # Проверю идентификаторы
        #
        
        if first_client.identifier.bytes != second_packet[:len(first_client.identifier.bytes)]:
            trace('[OtwayReesCA]', 'Malformed packet')
            return None
        
        if second_client.identifier.bytes != second_packet[len(first_client.identifier.bytes):]:
            trace('[OtwayReesCA]', 'Malformed packet')
            return None
        
        if N1 != N2:
            trace('[OtwayReesCA]', 'Malformed packet')
            return None
        
        #
        # Генерирую ключ и шифрую его на ключах пользователей
        #
        
        session_key = get_random_bytes(OtwayReesParams.SESSION_KEY_SIZE)
        
        cipher = AES.new(self._db[first_client.identifier], AES.MODE_CTR)
        first_nonce, first_packet = cipher.nonce, cipher.encrypt(Na + session_key)
        
        cipher = AES.new(self._db[second_client.identifier], AES.MODE_CTR)
        second_nonce, second_packet = cipher.nonce, cipher.encrypt(Nb + session_key)
        
        return first_nonce, first_packet, second_nonce, second_packet

In [17]:
class OtwayReesClient(object):
    def __init__(self, fake_data: uuid.UUID = None):
        """
        Инициализация клиента с созданием идентификатора и генерацией ключа.
        Создается пустая база для сессионных ключей и челленджей.
        Второй параметр требуется для демонстрации неуспешной попытки исполнения протокола.
        """
        
        self._id  = uuid.uuid4() if fake_data is None else fake_data
        self._key = get_random_bytes(NeedhamSchroederParams.ENCRYPTION_KEY_SIZE)
        
        self._db  = {}
        self._challenges = {}
        
        trace('[OtwayReesClient]', f'Client {self._id} created')
        
        
    def register(self, ca: OtwayReesCA):
        """
        Регистрация на ДЦ.
        """
        
        ca.register_client(self._id, self._key)
        
        
    def initiate_protocol(self, second_client, ca: OtwayReesCA) -> bool:
        """
        Инициация протокола транспортировки ключа. Принимает дескрипторы
        второго клиента и доверенного центра.
        Возвращает статус успешности выполнения протокола.
        """
        
        N  = get_random_bytes(OtwayReesParams.NUMBER_SIZE)
        Na = get_random_bytes(OtwayReesParams.NUMBER_SIZE)
        
        cipher = AES.new(self._key, AES.MODE_CTR)
        packet = cipher.encrypt(Na + N + self._id.bytes + second_client.identifier.bytes)
        
        #
        # Второй клиент принял запрос? Все ок?
        #
        
        encrypted_key = second_client._accept_initiation(N, self, cipher.nonce, packet, ca)
        if encrypted_key is None:
            trace('[OtwayReesClient]', f'[{self._id}] Second client denied packet')
            return False
        
        nonce, packet = encrypted_key
        
        cipher = AES.new(self._key, AES.MODE_CTR, nonce=nonce)
        packet = cipher.decrypt(packet)
        
        #
        # Проверяю то, что пришло с ДЦ.
        # Если все ок, то ключ установлен
        # 
        
        if Na != packet[:OtwayReesParams.NUMBER_SIZE]:
            trace('[OtwayReesClient]', f'[{self._id}] Malformed packet')
            return False
        
        self._db[second_client.identifier] = packet[OtwayReesParams.NUMBER_SIZE:]
        trace('[OtwayReesClient]', f'[{self._id}] Established key: {bytes_as_hex(self._db[second_client.identifier])}')
        
        return True
        
        
    def _accept_initiation(self, N: bytes, first_client, nonce: bytes, 
                           encrypted_packet: bytes, ca: OtwayReesCA):
        Nb = get_random_bytes(OtwayReesParams.NUMBER_SIZE)
        
        cipher = AES.new(self._key, AES.MODE_CTR)
        packet = cipher.encrypt(Nb + N + first_client.identifier.bytes + self._id.bytes)
        
        #
        # Засылаю полный пакет данных на сервер
        #
        
        result = ca.accept(N, first_client, self, nonce, encrypted_packet, cipher.nonce, packet)
        if result is None:
            trace('[OtwayReesClient]', f'[{self._id}] CA denied packets')
            return None
        
        first_nonce, first_packet, second_nonce, second_packet = result
        
        #
        # Проверю, что пришло то, что надо
        #
        
        cipher = AES.new(self._key, AES.MODE_CTR, nonce=second_nonce)
        packet = cipher.decrypt(second_packet)
        
        if Nb != packet[:OtwayReesParams.NUMBER_SIZE]:
            trace('[OtwayReesClient]', f'[{self._id}] Malformed packet')
            return None
        
        #
        # Ключ установлен
        #
        
        self._db[first_client.identifier] = packet[OtwayReesParams.NUMBER_SIZE:]
        trace('[OtwayReesClient]', f'[{self._id}] Established key: {bytes_as_hex(self._db[first_client.identifier])}')
        
        return first_nonce, first_packet
    
    
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора клиента.
        """
        
        return self._id

----

In [18]:
# Создаю доверенный центр
ca = OtwayReesCA()

In [19]:
# Два хороших пользователя
alice = OtwayReesClient()
bob   = OtwayReesClient()

[TRACE] [OtwayReesClient] Client dd8d2510-8eed-4fc9-b4b5-0afe03c45207 created
[TRACE] [OtwayReesClient] Client 6d685549-0900-402d-ae73-2097d2de5509 created


In [20]:
# Регистрация валидных пользователей
alice.register(ca)
bob.register(ca)

[TRACE] [OtwayReesCA] Attempting to register client dd8d2510-8eed-4fc9-b4b5-0afe03c45207 
        with key 68737e57b0c7b476a384bfe7a4dceea9fd98c1fb9add2b568d0c685f38404f88
[TRACE] [OtwayReesCA] Client registered successfully
[TRACE] [OtwayReesCA] Attempting to register client 6d685549-0900-402d-ae73-2097d2de5509 
        with key ed842a20e89ed562520b229ab29b495d9b204900c39586eb886161166cd7dbbf
[TRACE] [OtwayReesCA] Client registered successfully


In [21]:
# Протокол чудесно работает для валидных пользователей
alice.initiate_protocol(bob, ca)

[TRACE] [OtwayReesCA] Accepted initiation from dd8d2510-8eed-4fc9-b4b5-0afe03c45207 to 6d685549-0900-402d-ae73-2097d2de5509, N = 8dff436c
[TRACE] [OtwayReesClient] [6d685549-0900-402d-ae73-2097d2de5509] Established key: 7f30f7987e7695ee008f851fcec5632fa02d01cc1d9153eca3217516e48462f9
[TRACE] [OtwayReesClient] [dd8d2510-8eed-4fc9-b4b5-0afe03c45207] Established key: 7f30f7987e7695ee008f851fcec5632fa02d01cc1d9153eca3217516e48462f9


True

In [22]:
# А это злоумышленник
mallory = OtwayReesClient(bob.identifier)

[TRACE] [OtwayReesClient] Client 6d685549-0900-402d-ae73-2097d2de5509 created


In [23]:
# Он не может получить ключ, если влезет в общение вместо Боба
alice.initiate_protocol(mallory, ca)

[TRACE] [OtwayReesCA] Accepted initiation from dd8d2510-8eed-4fc9-b4b5-0afe03c45207 to 6d685549-0900-402d-ae73-2097d2de5509, N = 5d8e76f1
[TRACE] [OtwayReesCA] Malformed packet
[TRACE] [OtwayReesClient] [6d685549-0900-402d-ae73-2097d2de5509] CA denied packets
[TRACE] [OtwayReesClient] [dd8d2510-8eed-4fc9-b4b5-0afe03c45207] Second client denied packet


False

In [24]:
# Также он не сможет получить ключ, если сам инициирует протокол с Алисой
mallory.initiate_protocol(alice, ca)

[TRACE] [OtwayReesCA] Accepted initiation from 6d685549-0900-402d-ae73-2097d2de5509 to dd8d2510-8eed-4fc9-b4b5-0afe03c45207, N = 65466db5
[TRACE] [OtwayReesCA] Malformed packet
[TRACE] [OtwayReesClient] [dd8d2510-8eed-4fc9-b4b5-0afe03c45207] CA denied packets
[TRACE] [OtwayReesClient] [6d685549-0900-402d-ae73-2097d2de5509] Second client denied packet


False